In [4]:
import mysql.connector
import pickle
from jieba import analyse
import pickle
import os
import jieba
from jieba import analyse
from snownlp import SnowNLP
from snownlp.sentiment import Sentiment
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt

In [8]:
#输入：微博名
#输出： information：用户信息, senti：积极消极数, big5：大五人格得分以及评价,keywords：top50得词频
###information中各字段：'U_ID'：自动生成的ID； U_user_id', 用户ID；'U_name'真实姓名；'U_nick_name'用户昵称；'U_location'用户所在地；
#'U_sex'用户性别；'U_sexual_orientation'用户性取向；'U_Emotional_status'情感状态；'U_Birthday'生日；'U_constellation'星座；
#'U_Blood'血型；'U_own_bolg_address_name'个人微博地址；'U_brief_introduction'个人简介；'U_Registration_time'注册时间；'U_QQ'QQ
#'U_email'email;'U_MSN'MSN;'U_Work',工作信息;'U_educational'教育信息;'U_fans_Number'粉丝数;'U_blog_Number'博文数;"U_concerns_Number"关注数
#'U_credit'信誉度；
#'U_Head_Photo'数据库中的该字段用于存放图片的路径 是空的

def weibo_user(name):
    # 连接数据库
    coon = mysql.connector.connect(user='weibo_user', password='ir@DUT#weibo', host="10.8.128.205",
                                   port=23306)
    cursor = coon.cursor()
    cursor.execute('use weiboDB;')
    sql_str = 'select * from user_info_table4 where U_nick_name = "{0}";'.format(name)
    cursor.execute(sql_str)
    
    res = cursor.fetchall()
    for i in res:
        information = {"U_user_id":i[1],"U_name":i[2],"U_nick_name":i[3],"U_location":i[4],"U_sex":i[5],"U_sexual_orientation":i[6],
                 "U_Emotional_status":i[7],"U_Birthday":i[8],"U_constellation":i[9],"U_Blood":i[11],"U_own_bolg_address_name":i[12],
                 "U_brief_introduction":i[13],"U_Registration_time":i[14],"U_QQ":i[15],"U_email":i[16],"U_MSN":i[17],"U_Work":i[18],
                 "U_educational":i[19],"U_fans_Number":i[20],"U_concerns_Number":i[27],"U_blog_Number":i[21],"U_credit":i[22],"U_tag":i[28]}
        senti = {"U_senti":i[24]}      
        big = i[25][1:-1].split(",")
        dict_big5 = {'责任性':int(big[0]),'外倾性':int(big[1]),'开放性':int(big[2]),'神经质性':int(big[3]),'宜人性':int(big[4])}
        big5 = {"U_big5":dict_big5,"U_big5_comment":i[26]}
    sql_str1 =   'select W_content from weibo_table where W_nick_name = "{0}";'.format(name)
    cursor.execute(sql_str1)
    res = cursor.fetchall()
    tfidf = analyse.extract_tags
    key_word =[]
    for i in res: 
        keywords = tfidf(i[0],allowPOS=('ns','nr','nt','nz','nl','n', 'vn','vd','vg','v','vf','a','an','i'))
        result=[]
        for j in keywords:    
            result.append(j)
        key_word.append(" ".join(result)) 
    words = ""
    for line in key_word:
        for i in line:
            words = words+i
    result=jieba.analyse.textrank(words,topK=50,withWeight=True)
    keywords = dict()
    for i in result:
        keywords[i[0]]=i[1]
    cursor = coon.close()
    return information, senti, big5,keywords

In [9]:
information, senti, big5,keywords = weibo_user("谢磊NPU")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lulu\AppData\Local\Temp\jieba.cache
Loading model cost 0.689 seconds.
Prefix dict has been built succesfully.


In [10]:
information

{'U_user_id': '2627058773',
 'U_name': ' ',
 'U_nick_name': '谢磊NPU',
 'U_location': '陕西 西安',
 'U_sex': '男',
 'U_sexual_orientation': ' ',
 'U_Emotional_status': ' ',
 'U_Birthday': ' ',
 'U_constellation': ' ',
 'U_Blood': ' ',
 'U_own_bolg_address_name': 'https://weibo.com/xielei21st?from=inf&wvr=5&loc=infdomain',
 'U_brief_introduction': '西北工业大学计算机学院教授，从事智能语音处理、机器学习相关研究。研究兴趣：语音识别、语音合成、口语内容分析、音视频多模态处理等。',
 'U_Registration_time': datetime.datetime(2012, 2, 22, 0, 0),
 'U_QQ': ' ',
 'U_email': ' ',
 'U_MSN': ' ',
 'U_Work': ' ',
 'U_educational': '  学位：大学  学校： 西北工业大学 时间：2011年',
 'U_fans_Number': 3696,
 'U_concerns_Number': 412,
 'U_blog_Number': 9016,
 'U_credit': '信用较好',
 'U_tag': '                                     IT数码                                    '}

In [39]:
senti

{'U_senti': '[230, 11]'}

In [40]:
big5

{'U_big5': {'责任性': 13, '外倾性': 5, '开放性': 17, '神经质性': 13, '宜人性': 21},
 'U_big5_comment': '您的随和性得分最高，属于高随和性。随和性代表了“爱”，对合作和人际和谐是否看重。随和性高的人是善解人意的、友好的、慷慨大方的、乐于助人的，愿意为了别人放弃自己的利益。宜人性高的人对人性持乐观的态度，相信人性本善。宜人性低的人则把自己的利益放在别人的利益之上。本质上，他们不关心别人的利益，因此也不乐意去帮助别人。有时候，他们对别人是非常多疑的，怀疑别人的动机。对于某些职位来说，太高的宜人性是没有必要的，尤其是需要强硬和客观判断的场合，例如科学家、评论家和士兵。'}

In [41]:
keywords

{'转发': 1.0,
 '程序员': 0.05615517420046101,
 '小米': 0.05611426201611269,
 '可可': 0.054013417997516494,
 '生活': 0.04835218298882436,
 '浙大': 0.03603990426369304,
 '博士': 0.03409990252198326,
 '学术': 0.03233881234864703,
 '事儿': 0.02676341190433804,
 '出版': 0.024878436103641377,
 '大学': 0.023442270658064854,
 '组图': 0.022414312069188404,
 '测试': 0.02093249869606007,
 '功能': 0.02003314957760399,
 '看清': 0.019979084890502402,
 '美国': 0.01948095566798408,
 '勇气': 0.019343159381888637,
 '课程': 0.019165937635986053,
 '发现': 0.0190406871985275,
 '粉丝': 0.018763660291932514,
 '使用': 0.01813420077354424,
 '总裁': 0.018123578165085658,
 '图片': 0.018058622120344835,
 '原子弹': 0.0178730949251726,
 '地方': 0.01781017727018023,
 '问问': 0.017650447930957654,
 '中国': 0.017554379059716928,
 '语言': 0.017513151250951885,
 '知识分子': 0.01725124932034487,
 '责任': 0.017242278979910538,
 '审核': 0.01715230580685582,
 '还好': 0.01702505778814652,
 '教育': 0.016721609970158726,
 '听听': 0.016561910304580763,
 '科研': 0.016549654513895652,
 '总部': 0.01645914